In [18]:
# Set up a chain rule to calculate the loss function @ saturation w/out propagating derivatives through saturation solver
# This was done in the Language of Molecules paper.

# 1_differentiable_saft works when no solvers are involved, i.e. for properties specified with a given (V, T)
# if, instead, we want to solve for saturation conditions, then I have the old problem of 

In [19]:
import Pkg; Pkg.activate(".")

using Revise
using Clapeyron
includet("./saftvrmienn.jl")
# These are functions we're going to overload for SAFTVRMieNN
import Clapeyron: a_res, saturation_pressure

using Flux
using Plots
using ForwardDiff, DiffResults

using Zygote, ChainRulesCore
using ImplicitDifferentiation

  Activating project at `~/SAFT_ML`


In [26]:
function make_NN_model(Mw, m, σ, λ_a, λ_r, ϵ)
    model = SAFTVRMieNN(
        params = SAFTVRMieNNParams(
            Mw=[Mw],
            segment=[m],
            sigma=[σ],
            lambda_a=[λ_a],
            lambda_r=[λ_r],
            epsilon=[ϵ],
            epsilon_assoc=Float32[],
            bondvol=Float32[],
        )
    )
    return model
end

# Hack to get around Clapeyron model construction API
function make_model(Mw, m, σ, λ_a, λ_r, ϵ)
    model = SAFTVRMie(["methane"])
    
    model.params.Mw[1] = Mw
    model.params.segment[1] = m
    model.params.sigma[1] = σ
    model.params.lambda_a[1] = λ_a
    model.params.lambda_r[1] = λ_r
    model.params.epsilon[1] = ϵ

    return model
end

function forward_sat_p(X)
    T = 100.0
    model = make_model(X...)

    p, Vₗ, Vᵥ = saturation_pressure(model, T)
    
    return [p, Vₗ, Vᵥ]
end

# Saturation conditions:
#* At conditions specified by y, these two conditions are satisfied
#? But how do I turn this into an optimality condition with the same
#? dimension as X
function conditions_sat_p(X, y)
    T = 100.0
    # Mw, m, σ, λ_a, λ_r, ϵ = X
    model = make_NN_model(X...)
    
    p, Vₗ, Vᵥ = y
    
    obj = p + (a_res(model, Vᵥ, T) - a_res(model, Vₗ, T))/(1/Vᵥ - 1/Vₗ)
    return obj
end

impl_saturation_pressure = ImplicitFunction(forward_sat_p, conditions_sat_p)#; linear_solver=DirectLinearSolver())

ImplicitFunction(forward_sat_p, conditions_sat_p, IterativeLinearSolver(true, false), nothing)

In [33]:
X = [16.04, 1.0, 3.737e-10, 6.0, 12.504, 152.58]
# m = make_NN_model(X...)
# @show pressure(m, 3.695e-5, 100.0)
# m2 = make_model(X...)
# @show pressure(m2, 3.695e-5, 100.0)
y = impl_saturation_pressure(X)
conditions_sat_p(X, y)

34113.759878034776

In [29]:
g = Zygote.gradient(X -> conditions_sat_p(X, y), X)

(Union{Nothing, Float64}[nothing, -0.00019907032396560264, -19374.92189988039, 5.984904559933875e-5, 1.153376828529166e-5, -2.0198126408504992e-6],)

In [32]:
g[1]

6-element Vector{Union{Nothing, Float64}}:
       nothing
     -0.00019907032396560264
 -19374.92189988039
      5.984904559933875e-5
      1.153376828529166e-5
     -2.0198126408504992e-6

In [23]:
model = SAFTVRMie(["methane"])
saturation_pressure(model, 100.0)

(34113.7600622731, 3.6950451345479705e-5, 0.024068284363809818)

In [24]:
# Zygote.jacobian(impl_saturation_pressure, [16.04, 1.0, 3.737e-10, 6.0, 12.504, 152.58])
ForwardDiff.jacobian(impl_saturation_pressure, [16.04, 1.0, 3.737e-10, 6.0, 12.504, 152.58])
#! This doesn't seem to have worked... :(

MethodError: MethodError: no method matching a_res(::SAFTVRMieNN, ::Float64, ::Float64)
Closest candidates are:
  a_res(::SAFTVRMieNN, ::Any, ::Any, !Matched::Any) at ~/SAFT_ML/saftvrmienn.jl:161
  a_res(!Matched::Clapeyron.GEPCSAFTModel, ::Any, ::Any, !Matched::Any) at ~/.julia/dev/Clapeyron/src/models/SAFT/PCSAFT/variants/GEPCSAFT.jl:85
  a_res(!Matched::Clapeyron.CKSAFTModel, ::Any, ::Any, !Matched::Any) at ~/.julia/dev/Clapeyron/src/models/SAFT/CKSAFT/CKSAFT.jl:71
  ...

In [25]:
#* Instead, follow the example in Language of molecules paper, defining as the last newton step
